In [1]:
import sys
pt_models = "../input/pretrained-models/pretrained-models.pytorch-master/"
sys.path.insert(0, pt_models)
import pretrainedmodels

In [2]:
import glob
import torch
import albumentations
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image
import joblib
import torch.nn as nn
from torch.nn import functional as F

In [3]:
TEST_BATCH_SIZE=32
MODEL_MEAN=(0.485, 0.456, 0.406)
MODEL_STD=(0.229, 0.224, 0.225)
IMG_HEIGHT=137
IMG_WIDTH=236
DEVICE="cuda"

In [4]:
class ResNet50(nn.Module):
    def __init__(self, pretrained):
        super(ResNet50, self).__init__()
        if pretrained:
            self.model = pretrainedmodels.__dict__["resnet50"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet50"](pretrained=None)

        self.l0 = nn.Linear(2048, 168)
        self.l1 = nn.Linear(2048, 11)
        self.l2 = nn.Linear(2048, 7)

    def forward(self, x):
        bs, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        return l0, l1, l2

In [5]:
class BengaliDatasetTest:
    def __init__(self, df, img_height, img_width, mean, std):
        self.image_ids = df.image_id.values
        self.img_arr = df.iloc[:, 1:].values
        
        self.aug = albumentations.Compose([
                albumentations.Resize(img_height, img_width, always_apply=True),
                albumentations.Normalize(mean, std, always_apply=True)
            ])

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, item):
        image = self.img_arr[item, :]
        img_id = self.image_ids[item]
        
        image = image.reshape(137, 236).astype(float)
        image = Image.fromarray(image).convert("RGB")
        image = self.aug(image=np.array(image))["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        return{
            'image': torch.tensor(image, dtype=torch.float),
            'image_id': img_id
        }

In [6]:
# =======
def model_predict():
    g_pred, v_pred, c_pred = [], [], []
    img_ids_list = [] 
    
    for file_idx in range(4):
        df = pd.read_parquet(f"../input/bengaliai-cv19/test_image_data_{file_idx}.parquet")

        dataset = BengaliDatasetTest(df=df,
                                    img_height=IMG_HEIGHT,
                                    img_width=IMG_WIDTH,
                                    mean=MODEL_MEAN,
                                    std=MODEL_STD)

        data_loader = torch.utils.data.DataLoader(
            dataset=dataset,
            batch_size= TEST_BATCH_SIZE,
            shuffle=False,
            num_workers=4
        )

        for bi, d in enumerate(data_loader):
            image = d["image"]
            img_id = d["image_id"]
            image = image.to(DEVICE, dtype=torch.float)

            g, v, c = model(image)
            #g = np.argmax(g.cpu().detach().numpy(), axis=1)
            #v = np.argmax(v.cpu().detach().numpy(), axis=1)
            #c = np.argmax(c.cpu().detach().numpy(), axis=1)

            for ii, imid in enumerate(img_id):
                g_pred.append(g[ii].cpu().detach().numpy())
                v_pred.append(v[ii].cpu().detach().numpy())
                c_pred.append(c[ii].cpu().detach().numpy())
                img_ids_list.append(imid)
        
    return g_pred, v_pred, c_pred, img_ids_list

In [7]:
model = ResNet50(pretrained=False)
TEST_BATCH_SIZE = 32
Start_fold=0
final_g_pred = []
final_v_pred = []
final_c_pred = []
final_img_ids = []

for i in [Start_fold, 1, 3, 4]: # range(Start_fold, 5):
    model.load_state_dict(torch.load(f"../input/resnet50/resnet50_fold{i}.bin"))
#     print(model)
    model.to(DEVICE)
    model.eval()
    g_pred, v_pred, c_pred, img_ids_list = model_predict()
#     print(img_ids_list)
    final_g_pred.append(g_pred)
    final_v_pred.append(v_pred)
    final_c_pred.append(c_pred)
#     print(final_c_pred)
    if i == Start_fold:
        final_img_ids.extend(img_ids_list)

In [8]:
final_g = np.argmax(np.mean(np.array(final_g_pred), axis=0), axis=1)
final_v = np.argmax(np.mean(np.array(final_v_pred), axis=0), axis=1)
final_c = np.argmax(np.mean(np.array(final_c_pred), axis=0), axis=1)
# print(final_g)
# print(final_img_ids)
predictions = []
for ii, imid in enumerate(final_img_ids):

    predictions.append((f"{imid}_grapheme_root", final_g[ii]))
    predictions.append((f"{imid}_vowel_diacritic", final_v[ii]))
    predictions.append((f"{imid}_consonant_diacritic", final_c[ii]))

In [9]:
sub = pd.DataFrame(predictions,columns=["row_id","target"])
print(sub)

                         row_id  target
0          Test_0_grapheme_root       3
1        Test_0_vowel_diacritic       0
2    Test_0_consonant_diacritic       0
3          Test_1_grapheme_root      93
4        Test_1_vowel_diacritic       2
5    Test_1_consonant_diacritic       0
6          Test_2_grapheme_root      19
7        Test_2_vowel_diacritic       0
8    Test_2_consonant_diacritic       0
9          Test_3_grapheme_root     115
10       Test_3_vowel_diacritic       0
11   Test_3_consonant_diacritic       0
12         Test_4_grapheme_root      55
13       Test_4_vowel_diacritic       4
14   Test_4_consonant_diacritic       0
15         Test_5_grapheme_root     115
16       Test_5_vowel_diacritic       2
17   Test_5_consonant_diacritic       0
18         Test_6_grapheme_root     147
19       Test_6_vowel_diacritic       9
20   Test_6_consonant_diacritic       5
21         Test_7_grapheme_root     137
22       Test_7_vowel_diacritic       7
23   Test_7_consonant_diacritic       0


In [10]:
sub.to_csv("submission.csv",index=False)

In [11]:
# model = ResNet34(pretrained=False)
# model.load_state_dict(torch.load("../input/bengali-models/resnet34_fold0.bin"))
# model.eval()

In [12]:
# predictions = []
# for file_idx in range(4):
#     df = pd.read_parquet(f"../input/bengaliai-cv19/test_image_data_{file_idx}.parquet")
#     dataset = BengaliDatasetTest(df=df, img_height=IMG_HEIGHT, 
#                                  img_width=IMG_WIDTH, mean=MODEL_MEAN, std=MODEL_STD)
#     data_loader = torch.utils.data.DataLoader(dataset, 
#                                              batch_size=TEST_BATCH_SIZE,
#                                              shuffle=False,
#                                              num_workers=4)
#     for bi, d in enumerate(dataset):
#         image = d["image"]
#         img_id = d["image_id"]
#         image = image.to(DEVICE, dtype=torch.float)
#         g, v, c = model(image)
#         g = np.argmax(g, axis=1)
#         v = np.argmax(v, axis=1)
#         c = np.argmax(c, axis=1)
#         for ii, imid in enumerate(img_id):
#             predictions.append((f"{imid}_grapheme_root", g[ii]))
#             predictions.append((f"{imid}_vowel_diacritic", v[ii]))
#             predictions.append((f"{imid}_consonant_diacritic", c[ii]))

In [13]:
# sub = pd.DataFrame(predictions, columns=["row_id", "target"])

In [14]:
# sub.to_csv("submission.csv", index=False)